In [3]:
from scipy.stats import median_abs_deviation

import sys
sys.path.insert(0, '/home/workspace/mm_analysis')
sys.path.insert(0, '/home/workspace/')

from py_util import *
from utilities import *

hdir = '/home/workspace'
wdir = hdir + "/mm_analysis/EXP-01244/"
objdir = wdir + "quality_control/qc_objects/"

# Applying QC metrics

- BM starter was split into 3 10X barcodes due to availability of cells

In [9]:
cr_outs_path = os.path.join(hdir, "mm_analysis/EXP-01244/data/EXP-01244_cr_outs")

# Dictionary mapping user-friendly sample names to their corresponding IDs
sample_dict = {
   'week2': "OR07965-01",     # Maps time point labels to sample IDs
   'week3': "OR07965-02", 
   'week4': "OR00001",
   'bm': "BMC07965-007",      # Bone marrow sample
   'msc': "CELL00911"         # Mesenchymal stem cell sample
}

name_dict = {
   'week2': "Week 2",
   'week3': "Week 3", 
   'week4': "Week 4",
   'bm': "BMMC Start Sample",
   'msc': "MSC Start Sample"   
}

# Create reverse mapping from sample IDs to their user-friendly names
id_to_sample = {v: k for k, v in sample_dict.items()}

# Find all filtered_feature_bc_matrix.h5 files in the directory structure
h5_paths = [os.path.join(root, 'sample_filtered_feature_bc_matrix.h5') 
           for root, _, files in os.walk(cr_outs_path) 
           if 'sample_filtered_feature_bc_matrix.h5' in files]

# Get only the bm starter h5 paths
bm_paths = [path for path in h5_paths if "BMC07965-007" in path]

# Dictionary to store AnnData objects for each sample
adatas = {}

# Process each H5 file
for path in bm_paths:
    # Extract sample name from path
    name = path.split('per_sample_outs/')[1].split('/')[0]
    
    # Read the H5 file and create AnnData object
    adata = sc.read_10x_h5(path)
    adata.var_names_make_unique()

    adata.var["mt"] = adata.var_names.str.startswith("MT-")                 # mitochondrial genes
    adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))      # ribosomal genes
    adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))           # hemoglobin genes

    sc.pp.calculate_qc_metrics(
        adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
    )
    
    adata.obs["outlier"] = (
        is_outlier(adata, "log1p_total_counts", 5)                # calling outlier cells with log1p_total_counts greater than 5 MAD's
        | is_outlier(adata, "log1p_n_genes_by_counts", 5)         # calling outlier cells with log1p_n_genes_by_counts greater than 5 MAD's
        | is_outlier(adata, "pct_counts_in_top_20_genes", 5)      # calling outlier cells with pct_counts_in_top_20_genes greater than 5 MAD's
    )

    # Call mt outlier cells with pct_counts_mt greater than 3 MAD's
    adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3)

    # Run Scrublet for doublet detection
    scrub = scrublet.Scrublet(adata.X)
    doublet_scores, predicted_doublets = scrub.scrub_doublets(verbose=False)

    # Add Scrublet results to cell metadata
    adata.obs['doublet_score'] = doublet_scores
    adata.obs['predicted_doublet'] = predicted_doublets

    adata.obs['sample'] = name

    adata.obs['base_sample'] = adata.obs['sample'].str.replace(r'_\d+$', '', regex=True)      # Add metadata column for batched replicates
    adata.obs['sample_type'] = adata.obs['base_sample'].replace(id_to_sample)                 # Add sample names (week2, week3, etc.)
    adata.obs['name'] = adata.obs['sample_type'].replace(name_dict)                           # Add pretty names (Week 2, Week 3, etc.)

    adatas[name] = adata.copy()

bm_adata = ad.concat(adatas.values(), join='outer', merge='same')

# Store raw counts
bm_adata.layers["raw_counts"] = bm_adata.X.copy()

# Store log1p transformed normalized counts
scales_counts = sc.pp.normalize_total(bm_adata, copy=True)
bm_adata.layers["log1p_norm_counts"] = sc.pp.log1p(scales_counts.X, copy=True)

sc.pp.highly_variable_genes(bm_adata, layer="log1p_norm_counts")
sc.pp.pca(bm_adata, layer="log1p_norm_counts")
sc.pp.neighbors(bm_adata, n_neighbors=50, n_pcs=20)
sc.tl.umap(bm_adata)

bm_adata.write(objdir + 'bm_qc_adata.h5ad', compression='gzip')

... storing 'sample' as categorical
... storing 'base_sample' as categorical
... storing 'sample_type' as categorical
... storing 'name' as categorical
... storing 'feature_types' as categorical
... storing 'genome' as categorical
